In [11]:

import pandas as pd
import numpy as np
import json, os

# --------- Paramètres à adapter ---------
PATH = "reseau_en_arbre.xlsx"   
SHEET = 0                                  
OUT_DIR = "outputs_raccordement"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_ORDRE = os.path.join(OUT_DIR, "ordre_de_raccordement.csv")
OUT_TRAVAUX = os.path.join(OUT_DIR, "travaux_par_etape.csv")
OUT_KPIS  = os.path.join(OUT_DIR, "kpis.json")

# --------- Utils ----------
def std_cols(df):
    return df.rename(columns={c: str(c).strip().lower().replace(" ", "_") for c in df.columns})

def read_table(path, sheet=0):
    try:
        df = pd.read_excel(path, sheet_name=sheet)
    except Exception:
        df = pd.read_csv(path)
    return std_cols(df)

# --------- 1) Chargement & validation ----------
df = read_table(PATH, SHEET)

required = {"id_batiment", "infra_id", "longueur", "nb_maisons"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Colonnes manquantes {missing}. Requis: {sorted(required)}")

# Nettoyage minimal
df["id_batiment"] = df["id_batiment"].astype(str)
df["infra_id"]    = df["infra_id"].astype(str)
df["longueur"]    = pd.to_numeric(df["longueur"], errors="coerce").fillna(0.0)
df["nb_maisons"]  = pd.to_numeric(df["nb_maisons"], errors="coerce").fillna(1).astype(int)

# 2) Construction des objets 
# Liste unique des bâtiments et des infrastructures
batiments = (
    df.groupby("id_batiment", as_index=False)["nb_maisons"]
      .max()  # si plusieurs lignes pour un bâtiment, on garde nb_maisons max (ou ajuster selon ton fichier)
      .rename(columns={"nb_maisons":"prises"})
)
infras = (
    df.groupby("infra_id", as_index=False)["longueur"].max()
    # longueur d'une infra : on prend la longueur fournie (si plusieurs lignes/dup, on conserve max)
)

bat2infra = (
    df[["id_batiment","infra_id","longueur","nb_maisons"]]
    .copy()
)

# Ensemble des infras déjà réparées
repaired_infras = set()

# 3) Calcul difficulté 
def building_incremental_cost(b_id):
    rows = bat2infra[bat2infra["id_batiment"] == b_id]
    if rows.empty:
        return 0.0, []  # aucun travail
    nb = int(rows["nb_maisons"].iloc[0])
    # Infras encore non réparées
    rows = rows[~rows["infra_id"].isin(repaired_infras)]
    incr_cost = float((rows["longueur"] / max(1, nb)).sum())
    incr_infras = list(rows["infra_id"].unique())
    return incr_cost, incr_infras

# 4) Phase 0 
# Si rien n’indique des infras déjà OK, la phase 0 est vide (conforme plan strict)
ordre_rows = []
travaux_rows = []
etape = 0
cout_cumule = 0.0
prises_total = int(batiments["prises"].sum())
prises_cumulees = 0


# 5) Boucle greedy + mutualisation 
remaining = set(batiments["id_batiment"])
while remaining:
    # coût incrémental par bâtiment 
    candidates = []
    for b in remaining:
        cost, needed_infras = building_incremental_cost(b)
        candidates.append((b, cost, needed_infras))
    # choisir le plus facile (coût minimal, tie-break sur prises décroissantes puis id)
    bat_info = batiments.set_index("id_batiment")["prises"].to_dict()
    candidates.sort(key=lambda x: (x[1], -bat_info.get(x[0], 0), x[0]))
    b_best, cost_best, infras_best = candidates[0]

    etape += 1
    new_repairs = []
    for infra in infras_best:
        if infra not in repaired_infras:
            repaired_infras.add(infra)
            new_repairs.append(infra)

    cout_cumule += cost_best
    prises_cumulees += int(bat_info.get(b_best, 0))
    remaining.remove(b_best)

    ordre_rows.append({
        "rang": etape,
        "batiment": b_best,
        "cout_incremental": round(float(cost_best), 6),
        "cout_cumule": round(float(cout_cumule), 6),
        "prises_cumulees": int(prises_cumulees),
        "infras_reparees": new_repairs
    })

    for infra in new_repairs:
        long_infra = float(infras.loc[infras["infra_id"]==infra, "longueur"].max())
        travaux_rows.append({
            "etape": etape,
            "infra_id": infra,
            "longueur": long_infra,
            "type_intervention": "reparation"  
        })

# 6) KPI ----------
longueur_reparee = float(infras[infras["infra_id"].isin(repaired_infras)]["longueur"].sum())

# Taux de mutualisation 
# - coût_naif = somme pour chaque bâtiment de la somme des longueurs de SES infras (double-compte)
# - coût_reel = somme des longueurs des infras uniques réparées
cout_naif = float(
    bat2infra.groupby("id_batiment")["longueur"].sum().sum()
)
cout_reel = longueur_reparee
taux_mutualisation = None
if cout_naif > 0:
    taux_mutualisation = round(100.0 * (1.0 - cout_reel / cout_naif), 2)

kpis = {
    "cout_total_proxy": round(float(cout_reel), 6),          # ici le "coût" = longueur réparée totale
    "prises_total": int(prises_total),
    "prises_reconnectees": int(prises_cumulees),
    "taux_reconnexion_%": round(100.0 * prises_cumulees / prises_total, 2) if prises_total else None,
    "longueur_reparee_totale": round(float(longueur_reparee), 6),
    "taux_mutualisation_%": taux_mutualisation,
    "nb_etapes": int(etape)
}

# 7) Exports ----------
df_ordre = pd.DataFrame(ordre_rows)
df_travaux = pd.DataFrame(travaux_rows)

df_ordre.to_csv(OUT_ORDRE, index=False)
df_travaux.to_csv(OUT_TRAVAUX, index=False)
with open(OUT_KPIS, "w", encoding="utf-8") as f:
    json.dump(kpis, f, ensure_ascii=False, indent=2)

print("Fichiers :")
print(" -", OUT_ORDRE)
print(" -", OUT_TRAVAUX)
print(" -", OUT_KPIS)
print("KPIs :", kpis)

try:
    display(df_ordre.head(10))
    display(df_travaux.head(10))
except NameError:
    pass


Fichiers :
 - outputs_raccordement\ordre_de_raccordement.csv
 - outputs_raccordement\travaux_par_etape.csv
 - outputs_raccordement\kpis.json
KPIs : {'cout_total_proxy': 9208.059494, 'prises_total': 389, 'prises_reconnectees': 389, 'taux_reconnexion_%': 100.0, 'longueur_reparee_totale': 9208.059494, 'taux_mutualisation_%': 94.86, 'nb_etapes': 381}


,rang,batiment,cout_incremental,cout_cumule,prises_cumulees,infras_reparees
0,1,E000001,27.291663,27.291663,4,"[P007111, P007983, P000308, P007819]"
1,2,E000006,8.702344,35.994007,5,[P007173]
2,3,E000002,10.914626,46.908633,6,[P007240]
3,4,E000004,38.117331,85.025963,7,"[P000378, P007393]"
4,5,E000005,14.643782,99.669746,8,[P007249]
5,6,E000003,54.541389,154.211135,9,"[P007113, P007984, P007823]"
6,7,E000008,58.784250,212.995385,10,"[P007982, P000352, P007320]"
7,8,E000009,7.275098,220.270483,11,[P007286]
8,9,E000010,10.593967,230.864450,12,[P007115]
9,10,E000007,12.577324,243.441774,13,"[P000181, P007132]"


,etape,infra_id,longueur,type_intervention
0,1,P007111,12.314461,reparation
1,1,P007983,40.320929,reparation
2,1,P000308,39.140799,reparation
3,1,P007819,17.390464,reparation
4,2,P007173,8.702344,reparation
5,3,P007240,10.914626,reparation
6,4,P000378,27.481806,reparation
7,4,P007393,10.635525,reparation
8,5,P007249,14.643782,reparation
9,6,P007113,12.193509,reparation
